In [1]:
import torch
import numpy as np
import os
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

os.environ['MUJOCO_PY_MJPRO_PATH'] = '/sailhome/maxsobolmark/.mujoco/mjpro131'

pets_swimmer_path = '/sailhome/maxsobolmark/mbrl-lib/exp/fsrl/default/pets_swimmer_mbbl_lifelong_learning/2021.12.19/090259/'

In [2]:
replay_buffers = [np.load(f'{pets_swimmer_path}replay_buffer_task-{i}.npz') for i in range (4)]

In [3]:
from hydra.experimental import initialize, compose
initialize(config_path='mbrl/examples/conf')
cfg = compose(config_name='main_lifelong_learning', overrides=['overrides=pets_swimmer_mbbl_lifelong_learning', 'overrides.policy_mopo_penalty_coeff=0', 'overrides.planning_mopo_penalty_coeff=0'])
# cfg.device = 'cpu'

/iris/u/maxsobolmark/pytorch_mbbl/lib/python3.9/site-packages/hydra/core/utils.py:143: UserWarning: register_resolver() is deprecated.
See https://github.com/omry/omegaconf/issues/426 for migration instructions.

  OmegaConf.register_resolver(name, f)


In [4]:
from mbrl.examples.main_lifelong_learning import make_env
from mbrl.util.lifelong_learning import make_task_name_to_index_map
from mbrl.env.wrappers.multitask_wrapper import MultitaskWrapper
lifelong_learning_envs = []
lifelong_learning_task_names = []
lifelong_learning_termination_fns = []
lifelong_learning_reward_fns = []

# num_tasks = len(cfg.overrides.envs)

for i, env_cfg in enumerate(cfg.overrides.envs):
    env, term_fn, reward_fn = make_env(env_cfg.env_name, env_cfg.wrappers,
                                       cfg.overrides.learned_rewards, cfg,
                                       env_cfg)
    # env = MultitaskWrapper(env, i, num_tasks)
    lifelong_learning_envs.append(env)
    lifelong_learning_task_names.append(env_cfg.task_name)
    lifelong_learning_termination_fns.append(term_fn)
    lifelong_learning_reward_fns.append(reward_fn)

task_name_to_task_index = make_task_name_to_index_map(
    lifelong_learning_task_names)
num_tasks = len(task_name_to_task_index.keys())
lifelong_learning_envs = lifelong_learning_envs[:num_tasks]
lifelong_learning_termination_fns = (
    lifelong_learning_termination_fns[:num_tasks])
lifelong_learning_reward_fns = lifelong_learning_reward_fns[:num_tasks]
for i in range(len(lifelong_learning_envs)):
    lifelong_learning_envs[i] = MultitaskWrapper(
        lifelong_learning_envs[i], i, cfg.overrides.num_distinct_envs)

[2021-12-23 10:38:06,312] Making new env: Swimmer-v1


/iris/u/maxsobolmark/pytorch_mbbl/lib/python3.9/site-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)
[2021-12-23 10:38:06,329] Attempted to wrap env <mbbl.env.gym_env.walker.env object at 0x7f5b232f81f0> after .configure() was called.


[2021-12-23 10:38:06,330] Making new env: Swimmer-v1


[2021-12-23 10:38:06,339] Attempted to wrap env <mbbl.env.gym_env.walker.env object at 0x7f5ade7762e0> after .configure() was called.


[2021-12-23 10:38:06,341] Making new env: Swimmer-v1


[2021-12-23 10:38:06,350] Attempted to wrap env <mbbl.env.gym_env.walker.env object at 0x7f5ade7767f0> after .configure() was called.


[2021-12-23 10:38:06,352] Making new env: Swimmer-v1


[2021-12-23 10:38:06,362] Attempted to wrap env <mbbl.env.gym_env.walker.env object at 0x7f5ade776d30> after .configure() was called.


[main_lifelong_learning:168] reward_fn:  <function swimmer_mbbl at 0x7f5b1d2f1550>
[main_lifelong_learning:168] reward_fn:  <function swimmer_mbbl_backwards at 0x7f5b1d2f15e0>
[main_lifelong_learning:168] reward_fn:  <function swimmer_mbbl_forwards_up at 0x7f5b1d2f1670>
[main_lifelong_learning:168] reward_fn:  <function swimmer_mbbl_backwards_up at 0x7f5b1d2f1700>


In [5]:
import mbrl.util.common
from mbrl.models.lifelong_learning_model import LifelongLearningModel
obs_shape = lifelong_learning_envs[0].observation_space.shape
act_shape = lifelong_learning_envs[0].action_space.shape
dynamics_model = mbrl.util.common.create_one_dim_tr_model(cfg, obs_shape, act_shape)
dynamics_model = LifelongLearningModel(dynamics_model,
                                       cfg.overrides.num_distinct_envs,
                                       obs_shape,
                                       act_shape,
                                       cfg,
                                       observe_task_id=cfg.overrides.observe_task_id,
                                       forward_postprocess_fn=None)
import pickle
elite_models = pickle.load(open(pets_swimmer_path + 'elite_models.pkl', 'rb'))
dynamics_model.set_elite(elite_models)

[common:89] out_size:  8


/iris/u/maxsobolmark/pytorch_mbbl/lib/python3.9/site-packages/hydra/utils.py:32: UserWarning: `OmegaConf.is_none()` is deprecated, see https://github.com/omry/omegaconf/issues/547
  if OmegaConf.is_none(config):


self._model.no_delta_list:  []


In [10]:
from functools import partial
import mbrl.models
import mbrl.planning
from mbrl.util.lifelong_learning import (
    make_task_name_to_index_map, general_reward_function,
    train_lifelong_learning_model_and_save_model_and_data)
reward_function = partial(
    general_reward_function,
    list_of_reward_functions=lifelong_learning_reward_fns,
    device=cfg.device)
torch_generator = torch.Generator(device=cfg.device)
model_env = mbrl.models.ModelEnv(lifelong_learning_envs[0],
                                 dynamics_model,
                                 lifelong_learning_termination_fns[0],
                                 reward_function,
                                 generator=torch_generator)
planning_agent = mbrl.planning.create_trajectory_optim_agent_for_model(
    model_env,
    cfg.algorithm.agent,
    num_particles=cfg.algorithm.num_particles,
    planning_mopo_penalty_coeff=cfg.overrides.planning_mopo_penalty_coeff)

In [11]:
env = lifelong_learning_envs[0]
env.reset()
#
env.step(env.action_space.sample())
env.unwrapped._env.render(mode='rgb_array')

[2021-12-23 10:35:26,815] GLFW error: 65544, desc: b'X11: The DISPLAY environment variable is missing'


Exception: glfw failed to initialize

In [17]:
!pip install pyglet==1.3.2

     |████████████                    | 378 kB 10.6 MB/s eta 0:00:01

     |████████████████████████████████| 1.0 MB 10.6 MB/s 


  Attempting uninstall: pyglet
    Found existing installation: pyglet 1.5.0


    Uninstalling pyglet-1.5.0:


      Successfully uninstalled pyglet-1.5.0


In [8]:
planning_agent.reset()
obs = lifelong_learning_envs[0].reset()
total_reward = 0.
done = False
step = 0
while not done:
    print(f'Starting step {step}')
    action = planning_agent.act(obs)
    next_obs, reward, done, info = lifelong_learning_envs[0].step(action)
    total_reward += reward
    step += 1

Starting step 0


Starting step 1


Starting step 2


KeyboardInterrupt: 

test = lifelong_learning_envs[0].unwrapped._env.render(mode='rgb_array')